In [1]:
import matplotlib.patches
import numpy as np
import seaborn as sns
from functools import partial
from IPython.display import display
from ipywidgets import interact
from jupyter_renderer_widget import PyplotRenderer
from matplotlib import cm
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator
from scipy.special import factorial

In [2]:
def f(x):
    #return x ** 2 / 100
    return np.exp(np.sin(x) * 4)

x_vals = np.linspace(0, 10, 100)
y_vals = f(x_vals)

@interact(
    x1=(x_vals.min(), x_vals.max(), 0.1),
    h=(0.01, 5., 0.01),
)
def plot(
    x1=2.,
    h=4.,
):
    y1 = f(x1)    
    x2 = x1 + h
    y2 = f(x1 + h)
    slope = np.log(y2 / y1) / h
    #slope = (y2 - y1) / h
    #y3 = y2_prime * h + y1
    x4_vals = np.linspace(x1, x2, 50)
    y4_vals = np.exp((x4_vals - x1) * slope) * y1

    fig, ax = plt.subplots(1, 1, figsize=(14, 10))
    ax.plot(x_vals, y_vals)
    ax.plot(x1, y1, 'go')
    ax.plot(x2, y2, 'bo')
    #ax.plot(x2, y3, 'ro')
    #ax.plot([x1, x2], [y1, y3], 'r--')
    ax.plot(x4_vals, y4_vals, 'r--')

interactive(children=(FloatSlider(value=2.0, description='x1', max=10.0), FloatSlider(value=4.0, description='…

In [3]:
@interact(
    max_x=(0.01, 100, 0.01),
)
def f(
    max_x=4,
):
    x_vals = np.linspace(0, max_x, 100)
    y_vals = np.exp(x_vals) / np.exp(max_x)
    _, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.plot(x_vals, y_vals)
    ax.set_xlim(0, max_x)
    ax.set_ylim(0, 1)
    ax.set_title('Exponential asymptotic behavior when zooming out horizontally')

interactive(children=(FloatSlider(value=4.0, description='max_x', min=0.01, step=0.01), Output()), _dom_classe…

In [4]:
@interact(
    n=(1, 40, 1),
    x=(-4., 4., 0.01),
    y=(-8., 8., 0.01),
)
def f(
    n=25,
    x=0.,
    y=3.14,
    power_series=True,
    binom_series=False,
):
    xy = x + 1j * y
    terms1 = np.cumsum([xy ** i / factorial(i) for i in range(n)])
    terms2 = np.array([1] + [np.power(1 + xy / i, i) for i in range(1, n)])

    _, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.set_title('Exponential power & binomial series approximations')
    ax.set_xlim(-8., 8.)
    ax.set_ylim(-8., 8.)
    ax.set_aspect('equal')
    ax.axhline(0, color='k', ls='--', lw=0.5)
    ax.axvline(0, color='k', ls='--', lw=0.5)
    ax.grid()
    ax.plot([0, x], [0, y], 'r-')
    ax.plot(x, y, 'r*')
    if power_series:
        ax.plot(np.real(terms1), np.imag(terms1), 'go-', ms=4.)
    if binom_series:
        ax.plot(np.real(terms2), np.imag(terms2), 'bo-', ms=4.)
    ax.plot(np.real(np.exp(xy)), np.imag(np.exp(xy)), 'ro')

interactive(children=(IntSlider(value=25, description='n', max=40, min=1), FloatSlider(value=0.0, description=…

In [5]:
def quadratic_interpolate(t, t1=0.5, t2=None):
    if t2 is None:
        t2 = t1
    t1 = max(min(t1, 1.), 0.)
    t2 = max(min(t2, 1.), 0)
    if t1 + t2 > 1.:
        norm = t1 + t2
        t1 /= norm
        t2 /= norm
    c = 1 / (t2 * (2 - t2 - t1)) if t2 > 0. else 0.
    a = 1 / (t1 * (2 - t2 - t1)) if t1 > 0. else 0.
    b = a * t1 ** 2
    m = (1 - c * t2 ** 2 - b) / (1 - t2 - t1) if t2 + t1 < 1. else 0.
    return np.piecewise(
        t,
        [
            t < 0.,
            (t >= 0.) & (t < t1),
            (t >= t1) & (t < 1. - t2),
            (t >= 1. - t2) & (t < 1.),
            t >= 1.,
        ],
        [
            lambda t: np.zeros_like(t),
            lambda t: a * t ** 2,
            lambda t: m * (t - t1) + b,
            lambda t: 1 - c * (1 - t) ** 2,
            lambda t: np.ones_like(t),
        ]
    )


@interact(
    t1=(0., 1., 0.01),
    t2=(0., 1., 0.01),
)
def run(
    t1=0.25,
    t2=0.25,
):
    t = np.linspace(-0.1, 1.1, 100)
    y = quadratic_interpolate(t, t1, t2)
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    ax.plot(t, y)
    ax.set_title('Quadratic animation interpolator')
    ax.axhline(0)
    ax.axhline(1)
    ax.axvline(0)
    ax.axvline(1)
    ax.axvline(t1, ls='--', lw=1, alpha=0.6)
    ax.axvline(1 - t2, ls='--', lw=1, alpha=0.6)
    ax.set_ylim(-0.1, 1.1)

interactive(children=(FloatSlider(value=0.25, description='t1', max=1.0, step=0.01), FloatSlider(value=0.25, d…

In [6]:
def find_segment(segments, time):
    segment_map = {}
    start_time = 0.
    result = None
    if time < 0.:
        raise IndexError('Invalid negative time: {:2.f}')
    for duration, func in segments:
        if time >= start_time and time < start_time + duration:
            result = start_time, duration, func
            break
        start_time += duration
    else:
        raise IndexError('Time is past total duration {:.2f}: {:.2f}'.format(start_time, time))
    return result


def get_total_duration(segments):
    time = 0.
    for duration, _ in segments:
        time += duration
    return time

In [7]:
FPS = 25.
GRID_SIZE_X = 6
GRID_SIZE_Y = 2 * np.pi
GRID_MAJOR = 60
GRID_MINOR = 5
GRID_COUNT = GRID_MAJOR * GRID_MINOR + 1
MIN_SCALE = 1.
MAX_SCALE = 1.

x_vals = np.linspace(-GRID_SIZE_X, GRID_SIZE_X, GRID_COUNT)
y_vals = np.linspace(-GRID_SIZE_Y, GRID_SIZE_Y, GRID_COUNT)
x_vals = np.where(np.isclose(x_vals, 0), np.zeros_like(x_vals), x_vals)
y_vals = np.where(np.isclose(y_vals, 0), np.zeros_like(y_vals), y_vals)
x_grid, y_grid = np.meshgrid(x_vals, y_vals)
xy_grid = (x_grid + y_grid * 1j)
xy_vals = xy_grid.ravel()


segments = [
    (0.5, lambda t: 0.),
    (4.5, lambda t: quadratic_interpolate(t, 0.4)),
    (0.5, lambda t: 1.),
    (4.5, lambda t: 1 - quadratic_interpolate(t, 0.4)),
]
total_duration = get_total_duration(segments)
frame_count = int(total_duration * FPS)


def render(ax, frame_num):
    time = frame_num / FPS
    start_time, duration, func = find_segment(segments, time)
    t = func((time - start_time) / duration)
    #t = quadratic_interpolate(0.25, 0.25)

    #z_vals = xy_vals ** (1 + t)
    with np.errstate(all='ignore'):
        #z_grid = (1 - t) * xy_grid + t * np.exp(t * xy_grid + (1 - t) * np.log(xy_grid))
        z_grid = np.exp(t * xy_grid + (1 - t) * np.log(xy_grid))
        #z_grid = np.where(
        #    np.isclose(xy_grid.imag, 0),
        #    np.exp(t * xy_grid),  # + (1 - t) * xy_grid,
        #    t * xy_grid + (1 - t) * np.exp(t * xy_grid + (1 - t) * np.log(xy_grid)),
        #)
    #z_grid_interp = xy_grid * (1. - t) + z_grid * t
    z_grid_interp = z_grid
    z_vals_interp = z_grid_interp.ravel()
    ax.set_title('Exponential function on the complex plane: a+bi -> e^(a+bi)')
    #ax.set_title('X^4 transformation on complex plane')
    scale = MIN_SCALE
    if MAX_SCALE - MIN_SCALE > 0.:
        scale *= np.exp(np.log(MAX_SCALE / MIN_SCALE) * t)
    ax.set_xlim(-10. * scale, 10. * scale)
    ax.set_ylim(-10. * scale, 10. * scale)
    ax.set_aspect('equal')
    ax.axhline(0, color='k', ls='--', lw=0.5)
    ax.axvline(0, color='k', ls='--', lw=0.5)
    ax.xaxis.set_major_locator(MultipleLocator(10))
    ax.yaxis.set_major_locator(MultipleLocator(10))
    if scale < 6:
        ax.xaxis.set_minor_locator(AutoMinorLocator(10))
        ax.yaxis.set_minor_locator(AutoMinorLocator(10))
    ax.grid()
    ax.set_xlabel('real (a)')
    ax.set_ylabel('imaginary (bi)')
    
    z_points = z_grid_interp[::GRID_MINOR, ::GRID_MINOR].ravel()
    ax.plot(z_points.real, z_points.imag, 'o', ms=2, alpha=0.6)
    for i in range(0, z_grid.shape[0], GRID_MINOR):
        vals = z_grid_interp[i, :]
        y = xy_grid[i, 0].imag
        alpha = 0.5
        lw = 0.6
        color = 'b'
        if np.isclose(y, 0):
            # Hack to handle bifurcation for x<0 y=0 due to log ambiguity.
            ax.plot(vals.real, -vals.imag, '-', color=color, lw=lw, alpha=alpha)
        elif np.isclose(y, np.pi):
            color = 'r'
            alpha = 1.
            lw = 1.5
        elif np.isclose(y, -np.pi):
            color = 'purple'
            alpha = 1.
            lw = 1.5
        ax.plot(vals.real, vals.imag, '-', color=color, lw=lw, alpha=alpha)

    for i in range(0, z_grid.shape[1], GRID_MINOR):
        vals = z_grid_interp[:, i]
        x = xy_grid[0, i].real
        alpha = 0.5
        lw = 0.6
        color = 'b'
        if np.isclose(x, 0.):
            color = 'lightgreen'
            alpha = 1.
            lw = 2
        ax.plot(vals.real, vals.imag, '-', color=color, lw=lw, alpha=alpha)

    ax.legend(
        handles=[
            matplotlib.patches.Patch(color='lightgreen', label='a = 0'),
            matplotlib.patches.Patch(color='r', label='b = π'),
            matplotlib.patches.Patch(color='purple', label='b = -π'),
        ],
        loc='lower left',
    )


if False:
    @interact(
        t=(0., 1., 0.01),
    )
    def f(
        t=1.,
    ):
        _, ax = plt.subplots(1, 1, figsize=(12, 8))
        render(ax, t * frame_count)

PyplotRenderer(render, frame_count=frame_count, width=800, height=800)

PyplotRenderer(children=(Button(description='Render', style=ButtonStyle()), Output()))

In [47]:
FPS = 25
GRID_SIZE_X = 6
GRID_SIZE_Y = 6
GRID_MAJOR = 60
GRID_MINOR = 20
GRID_COUNT = GRID_MAJOR * GRID_MINOR + 1
MIN_SCALE = 0.45
MAX_SCALE = 0.45  #10.

x_vals = np.linspace(-GRID_SIZE_X, GRID_SIZE_X, GRID_COUNT)
y_vals = np.linspace(-GRID_SIZE_Y, GRID_SIZE_Y, GRID_COUNT)
x_vals = np.where(np.isclose(x_vals, 0), np.zeros_like(x_vals), x_vals)
y_vals = np.where(np.isclose(y_vals, 0), np.zeros_like(y_vals), y_vals)
x_grid, y_grid = np.meshgrid(x_vals, y_vals)
xy_grid = (x_grid + y_grid * 1j)
xy_vals = xy_grid.ravel()

with np.errstate(divide='ignore'):
    w_grid = 30 * np.log(np.abs(np.sqrt(x_grid ** 2 + y_grid ** 2) - 1))
w_grid = np.where(w_grid < -100, np.zeros_like(w_grid), w_grid)
w_grid += 8 * np.sin(8 * np.arctan2(y_grid, x_grid))
w_grid -= w_grid.min()
w_grid /= w_grid.max()
w_grid = (w_grid * 6.) % 1

@interact(
    zoom=(1., 5., 0.1)
)
def f(zoom=1.):
    scale = 1 / zoom
    _, ax = plt.subplots(1, 1)
    ax.pcolormesh(x_grid, y_grid, w_grid, cmap='hsv')
    ax.set_title('Depth map (w)')
    ax.set_xlim(x_vals.min() * scale, x_vals.max() * scale)
    ax.set_ylim(x_vals.min() * scale, x_vals.max() * scale)

interactive(children=(FloatSlider(value=1.0, description='zoom', max=5.0, min=1.0), Output()), _dom_classes=('…

In [50]:
if False:
    segments = [
        (0.5, lambda t: 0.),
        (3., lambda t: quadratic_interpolate(t, 0.3)),
        (0.5, lambda t: 1.),
        (4., lambda t: 1. + quadratic_interpolate(t, 0.3)),
        (0.5, lambda t: 2.),
        (4., lambda t: 2. + quadratic_interpolate(t, 0.3)),
        (0.5, lambda t: 3.),
        (6., lambda t: 3. * (1. - quadratic_interpolate(t, 0.3))),
    ]
else:
    segments = [
        (0.5, lambda t: 0.),
        (3., lambda t: quadratic_interpolate(t, 0.3)),
        (0.5, lambda t: 1.),
        (4., lambda t: 1. + quadratic_interpolate(t, 0.3)),
        (0.5, lambda t: 2.),
        (4., lambda t: 2. + quadratic_interpolate(t, 0.3)),
        (0.5, lambda t: 3.),
        (4., lambda t: 3. + quadratic_interpolate(t, 0.3)),
        (0.5, lambda t: 4.),
        (4., lambda t: 4. + quadratic_interpolate(t, 0.3)),
        (0.5, lambda t: 5.),
        (4., lambda t: 5. + quadratic_interpolate(t, 0.3)),
        (0.5, lambda t: 6.),
        (4., lambda t: 6. + quadratic_interpolate(t, 0.3)),
        (0.5, lambda t: 7.),
        (4., lambda t: 7. + quadratic_interpolate(t, 0.3)),
        (0.5, lambda t: 8.),
        (4., lambda t: 8. + quadratic_interpolate(t, 0.3)),
        (2.5, lambda t: 9.),
        (8., lambda t: 9. * (1. - quadratic_interpolate(t, 0.3))),
    ]
total_duration = get_total_duration(segments)
frame_count = int(total_duration * FPS)


def render(ax, frame_num):
    time = frame_num / FPS
    start_time, duration, func = find_segment(segments, time)
    t = func((time - start_time) / duration)

    z_grid = xy_grid ** (1 + t)
    z_grid_interp = z_grid
    z_vals_interp = z_grid_interp.ravel()
    ax.set_title('(a+bi)^{:.2f}'.format(t + 1))
    scale = MIN_SCALE
    if MAX_SCALE - MIN_SCALE > 0.:
        scale *= np.exp(np.log(MAX_SCALE / MIN_SCALE) * t / 9)
    ax.set_xlim(-10. * scale, 10. * scale)
    ax.set_ylim(-10. * scale, 10. * scale)
    ax.set_aspect('equal')
    ax.axhline(0, color='k', ls='--', lw=0.5)
    ax.axvline(0, color='k', ls='--', lw=0.5)
    if scale < 5:
        major_ticks = 10
    elif scale < 15:
        major_ticks = 10
    else:
        major_ticks = 100
    ax.xaxis.set_major_locator(MultipleLocator(major_ticks))
    ax.yaxis.set_major_locator(MultipleLocator(major_ticks))
    if scale < 6:
        ax.xaxis.set_minor_locator(AutoMinorLocator(10))
        ax.yaxis.set_minor_locator(AutoMinorLocator(10))
    #ax.grid()
    ax.set_xlabel('real (a)')
    ax.set_ylabel('imaginary (bi)')
    
    ax.pcolormesh(
        z_grid.real,
        z_grid.imag,
        w_grid,
        alpha=0.2,
        edgecolors='none',
        shading='gouraud',
        cmap='hsv',
    )
    
    z_points = z_grid_interp[::GRID_MINOR, ::GRID_MINOR].ravel()
    ax.plot(z_points.real, z_points.imag, 'o', ms=1.5, alpha=0.6)
    for i in range(0, z_grid.shape[0], GRID_MINOR):
        vals = z_grid_interp[i, :]
        y = xy_grid[i, 0].imag
        alpha = 0.7
        lw = 3.
        style = '-'
        if np.isclose(y, 2.):
            color = 'red'
        elif np.isclose(y, 1.):
            color = 'orange'
        elif np.isclose(y, 0.):
            color = 'yellow'
        elif np.isclose(y, -1.):
            color = 'green'
        elif np.isclose(y, -2.):
            color = 'blue'
        else:
            alpha = 0.3
            lw = 0.6
            color = 'b'
            style = '-'
        ax.plot(vals.real, vals.imag, style, color=color, lw=lw, alpha=alpha)

    for i in range(0, z_grid.shape[1], GRID_MINOR):
        vals = z_grid_interp[:, i]
        x = xy_grid[0, i].real
        alpha = 0.7
        lw = 3.
        style = '-'
        if np.isclose(x, 0.):
            color = 'purple'
        elif np.isclose(x, 1.):
            color = 'violet'
        else:
            alpha = 0.3
            lw = 0.6
            color = 'b'
            style = '-'
        ax.plot(vals.real, vals.imag, style, color=color, lw=lw, alpha=alpha)

    ax.legend(
        handles=[
            matplotlib.patches.Patch(color='red', label='a + 2i'),
            matplotlib.patches.Patch(color='orange', label='a + 1i'),
            matplotlib.patches.Patch(color='yellow', label='a + 0i'),
            matplotlib.patches.Patch(color='green', label='a - 1i'),
            matplotlib.patches.Patch(color='blue', label='a - 2i'),
            matplotlib.patches.Patch(color='purple', label='0 + bi'),
            matplotlib.patches.Patch(color='violet', label='1 + bi'),
        ],
        loc='lower left',
    )


if False:
    @interact(
        t=(0., 1., 0.01),
    )
    def f(
        t=1.,
    ):
        _, ax = plt.subplots(1, 1, figsize=(12, 8))
        render(ax, t * frame_count)

#frame_count = 50
PyplotRenderer(render, frame_count=frame_count, width=800, height=800)

PyplotRenderer(children=(Button(description='Render', style=ButtonStyle()), Output()))

In [ ]:
@interact(
    b_re=(-10, 10, 0.01),
    b_im=(-10, 10, 0.01),
    x_re=(-10, 10, 0.01),
    x_im=(-10, 10, 0.01),
)
def run(
    b_re=np.e,
    b_im=0.,
    x_re=0.,
    x_im=np.pi,
):
    x = x_re + x_im * 1j
    b = b_re + b_im * 1j
    y = (np.arange(-5, 6) * 2 * np.pi + b_im) * 1j + np.log(b)
    z = x * y
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    ax.plot(x.real, x.imag, 'bo', alpha=0.5)
    ax.plot(y.real, y.imag, 'go', alpha=0.5)
    ax.plot(z.real, z.imag, 'ro', alpha=0.5)
    ax.set_xlim(-15, 15)
    ax.set_ylim(-15, 15)
    ax.set_aspect('equal')
    ax.axhline(0, color='k', lw=1)
    ax.axvline(0, color='k', lw=1)
    ax.grid()